# Kerasレイヤーを作成

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# ダミーデータ生成
import numpy as np
x_train = np.random.random((1000, 20))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
x_test = np.random.random((100, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

model = Sequential()
# Dense(64) は，64個のhidden unitを持つ全結合層です．
# 最初のlayerでは，想定する入力データshapeを指定する必要があり，ここでは20次元としてます．
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))




シンプルで状態を持たない独自演算では，layers.core.Lambdaを用いるべきでしょう．     
しかし，学習可能な重みを持つ独自演算は，自身でレイヤーを実装する必要があります．    

以下にKeras 2.0でのレイヤーの枠組みを示します（古いバージョンを使っている場合は，更新してください）． 実装する必要のあるメソッドは3つだけです．    

- build(input_shape): これは重みを定義するメソッドです．このメソッドは，self.built = Trueをセットしなければいけません，これはsuper([Layer], self).build()を呼び出しでできます．

- call(x): ここではレイヤーのロジックを記述します．オリジナルのレイヤーでマスキングをサポートしない限り，第1引数である入力テンソルがcallに渡されることに気を付けてください．

- compute_output_shape(input_shape): 作成したレイヤーの内部で入力のshapeを変更する場合には，ここでshape変換のロジックを指定する必要があります．こうすることでKerasが自動的にshapeを推定します．

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np

class MyLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
    
    #重みを定義
    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this somewhere!
    #入力テンソルがここに渡される
    def call(self, x):
        #内積取るだけのレイヤー
        return K.dot(x, self.kernel)
    #出力のshapeを変更したい場合は定義する
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [0]:
model.add(MyLayer(10))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [7]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/20
1000/1000 [==============================] - 1s 1ms/step - loss: 2.3097 - acc: 0.0910
Epoch 2/20
1000/1000 [==============================] - 0s 47us/step - loss: 2.3008 - acc: 0.1060
Epoch 3/20
1000/1000 [==============================] - 0s 44us/step - loss: 2.2998 - acc: 0.1050
Epoch 4/20
1000/1000 [==============================] - 0s 44us/step - loss: 2.3012 - acc: 0.1160
Epoch 5/20
1000/1000 [==============================] - 0s 53us/step - loss: 2.3000 - acc: 0.1070
Epoch 6/20
1000/1000 [==============================] - 0s 48us/step - loss: 2.3031 - acc: 0.1150
Epoch 7/20
1000/1000 [==============================] - 0s 49us/step - loss: 2.2987 - acc: 0.1180
Epoch 8/20
1000/1000 [==============================] - 0s 45us/step - loss: 2.3011 - acc: 0.1120
Epoch 9/20
1000/1000 [==============================] - 0s 49us/step - loss: 2.3033 - acc: 0.1060
Epoch 10/20
1000/1000 [==============================] - 0s 52us/step - loss: 2.2990 - acc: 0.1040
Epoch 11/20
1000/100